In [1]:
import pandas as pd

In [10]:
age_raw = pd.read_csv("../data/census_raw/age/ACSDT5Y2021.B01002-Data.csv", skiprows=1)
age = age_raw.loc[:, ['Geography', 'Geographic Area Name', 'Estimate!!Median age --!!Total:']]
age.columns = ['geoid', 'name', 'median_age']
age

,geoid,name,median_age
0,1500000US510010901011,"Block Group 1, Census Tract 901.01, Accomack C...",62.8
1,1500000US510010901012,"Block Group 2, Census Tract 901.01, Accomack C...",50.3
2,1500000US510010901021,"Block Group 1, Census Tract 901.02, Accomack C...",67.0
3,1500000US510010901022,"Block Group 2, Census Tract 901.02, Accomack C...",30.3
4,1500000US510010902011,"Block Group 1, Census Tract 902.01, Accomack C...",53.3
...,...,...,...
5958,1500000US518400003014,"Block Group 4, Census Tract 3.01, Winchester c...",30.9
5959,1500000US518400003021,"Block Group 1, Census Tract 3.02, Winchester c...",30.6
5960,1500000US518400003022,"Block Group 2, Census Tract 3.02, Winchester c...",47.6
5961,1500000US518400003023,"Block Group 3, Census Tract 3.02, Winchester c...",36.2


In [53]:
emp_raw = pd.read_csv("../data/census_raw/employment/ACSDT5Y2021.B23025-Data.csv", skiprows=1)
emp_raw.loc[:, 'unemploy_pct'] = emp_raw['Estimate!!Total:!!In labor force:!!Civilian labor force:!!Unemployed'] / emp_raw[
    'Estimate!!Total:!!In labor force:!!Civilian labor force:'] * 100
emp = emp_raw.loc[:, ['Geography', 'unemploy_pct']]
emp.columns = ['geoid', 'unemploy_pct']
emp

,geoid,unemploy_pct
0,1500000US510010901011,0.000000
1,1500000US510010901012,14.913958
2,1500000US510010901021,12.970711
3,1500000US510010901022,0.000000
4,1500000US510010902011,6.435644
...,...,...
5958,1500000US518400003014,1.862464
5959,1500000US518400003021,2.444703
5960,1500000US518400003022,6.595538
5961,1500000US518400003023,5.693431


In [32]:
income_raw = pd.read_csv("../data/census_raw/income/ACSDT5Y2021.B19013-Data.csv", skiprows=1)
income = income_raw.loc[:, ['Geography', 'Estimate!!Median household income in the past 12 months (in 2021 inflation-adjusted dollars)']]
income.columns = ['geoid', 'median_income']
income

,geoid,median_income
0,1500000US510010901011,60026
1,1500000US510010901012,67121
2,1500000US510010901021,43929
3,1500000US510010901022,23990
4,1500000US510010902011,31606
...,...,...
5958,1500000US518400003014,-
5959,1500000US518400003021,37105
5960,1500000US518400003022,109412
5961,1500000US518400003023,88500


In [52]:
education_raw = pd.read_csv("../data/census_raw/education/ACSDT5Y2021.B15003-Data.csv", skiprows=1)
education = pd.DataFrame()
education.loc[:, 'geoid'] = education_raw['Geography']
education_raw.loc[:, 'college_degree'] = education_raw.loc[:, ['Estimate!!Total:!!Associate\'s degree',
       'Estimate!!Total:!!Bachelor\'s degree',
       'Estimate!!Total:!!Master\'s degree',
       'Estimate!!Total:!!Professional school degree',
       'Estimate!!Total:!!Doctorate degree']].sum(axis=1)
education_raw.loc[:, 'total_pop'] = education_raw[[col for col in education_raw.columns if col.startswith("Estimate")]].sum(axis=1)
education.loc[:, 'college_degree_pct'] = education_raw.loc[:, 'college_degree'] / education_raw.loc[:, 'total_pop'] * 100
education

,geoid,college_degree_pct
0,1500000US510010901011,24.751861
1,1500000US510010901012,23.396675
2,1500000US510010901021,24.516575
3,1500000US510010901022,24.920635
4,1500000US510010902011,10.188088
...,...,...
5958,1500000US518400003014,9.587379
5959,1500000US518400003021,16.845494
5960,1500000US518400003022,33.167559
5961,1500000US518400003023,19.151671


In [57]:
race_raw = pd.read_csv("../data/census_raw/race/ACSDT5Y2021.B02001-Data.csv", skiprows=1)
race = pd.DataFrame()
race.loc[:, 'geoid'] = race_raw['Geography']
race.loc[:, 'white_pct'] = race_raw['Estimate!!Total:!!White alone'] / race_raw['Estimate!!Total:'] * 100
race

,geoid,white_pct
0,1500000US510010901011,99.106145
1,1500000US510010901012,94.311377
2,1500000US510010901021,98.614610
3,1500000US510010901022,86.363636
4,1500000US510010902011,77.892031
...,...,...
5958,1500000US518400003014,80.674342
5959,1500000US518400003021,73.770492
5960,1500000US518400003022,76.758242
5961,1500000US518400003023,88.027108


In [63]:
from functools import reduce
census_data_merged = reduce(lambda left, right: pd.merge(left, right, on='geoid', how='outer'),
                           [age, race, income, education, emp])
for col in census_data_merged.columns[2:]:
    census_data_merged[col] = pd.to_numeric(census_data_merged[col], errors='coerce')
census_data_merged

,geoid,name,median_age,white_pct,median_income,college_degree_pct,unemploy_pct
0,1500000US510010901011,"Block Group 1, Census Tract 901.01, Accomack C...",62.8,99.106145,60026.0,24.751861,0.000000
1,1500000US510010901012,"Block Group 2, Census Tract 901.01, Accomack C...",50.3,94.311377,67121.0,23.396675,14.913958
2,1500000US510010901021,"Block Group 1, Census Tract 901.02, Accomack C...",67.0,98.614610,43929.0,24.516575,12.970711
3,1500000US510010901022,"Block Group 2, Census Tract 901.02, Accomack C...",30.3,86.363636,23990.0,24.920635,0.000000
4,1500000US510010902011,"Block Group 1, Census Tract 902.01, Accomack C...",53.3,77.892031,31606.0,10.188088,6.435644
...,...,...,...,...,...,...,...
5958,1500000US518400003014,"Block Group 4, Census Tract 3.01, Winchester c...",30.9,80.674342,NaN,9.587379,1.862464
5959,1500000US518400003021,"Block Group 1, Census Tract 3.02, Winchester c...",30.6,73.770492,37105.0,16.845494,2.444703
5960,1500000US518400003022,"Block Group 2, Census Tract 3.02, Winchester c...",47.6,76.758242,109412.0,33.167559,6.595538
5961,1500000US518400003023,"Block Group 3, Census Tract 3.02, Winchester c...",36.2,88.027108,88500.0,19.151671,5.693431


In [65]:
census_data_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5963 entries, 0 to 5962
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   geoid               5963 non-null   object 
 1   name                5963 non-null   object 
 2   median_age          5909 non-null   float64
 3   white_pct           5914 non-null   float64
 4   median_income       5554 non-null   float64
 5   college_degree_pct  5911 non-null   float64
 6   unemploy_pct        5902 non-null   float64
dtypes: float64(5), object(2)
memory usage: 326.2+ KB


In [66]:
census_data_merged.to_csv("../data/census_data_merged.csv")